In [ ]:
# 実行後、ランタイムを再起動してください
!pip install dwave-ocean-sdk
!pip install pyqubo

In [ ]:
import numpy as np

# 都市数
N = 5

# 都市間の移動距離
Q = np.array([[1000, 20, 20, 50, 40],
              [30, 1000, 10, 30, 20],
              [20, 10, 1000, 30, 20],
              [50, 30, 20, 1000, 10],
              [40, 20, 20, 10, 1000]])

In [ ]:
# 変数を作成
from pyqubo import Array, Constraint, Placeholder
x = Array.create('x', shape=(N, N), vartype='BINARY')

In [ ]:
# コスト関数
cost = 0
for t in range(N):
  for i in range(N):
    for j in range(N):
      cost += Q[i][j]*x[t][i]*x[(t+1)%N][j]

In [ ]:
# 制約条件1：各都市に1回は訪問しなければならない
constr_1 = 0
for i in range(N):
  constr_1 += (np.sum(x[i])-1)**2

# 制約条件2：1度に訪れる都市は1つでなければならない
constr_2 = 0
for i in range(N):
  constr_2 += (np.sum(x.T[i])-1)**2

In [ ]:
cost_func = cost + Placeholder('lam')*Constraint(constr_1, label='constr_1')
                 + Placeholder('lam')*Constraint(constr_2, label='constr_2')
model = cost_func.compile()

In [ ]:
feed_dict = {'lam': 1000} # penalty係数
qubo, offset = model.to_qubo(feed_dict=feed_dict)

# D-Waveマシンでの実行

In [ ]:
token = "D-Wave Leapのtokenを入力してください"
endpoint = 'https://cloud.dwavesys.com/sapi/'

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
dw_sampler = DWaveSampler(solver='DW_2000Q_6', token=token, endpoint=endpoint)

In [ ]:
sampler = EmbeddingComposite(dw_sampler)

In [ ]:
sampleset = sampler.sample_qubo(qubo, num_reads=10)

In [ ]:
# 結果出力
print(sampleset.record)

[([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], -9910., 1, 0.)
 ([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], -9890., 1, 0.)
 ([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1], -7890., 1, 0.)
 ([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], -7870., 1, 0.)
 ([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], -7870., 1, 0.)
 ([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], -7850., 1, 0.)
 ([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], -7850., 1, 0.)
 ([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], -6860., 1, 0.)
 ([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], -5820., 1, 0.)
 ([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1], -5790., 1, 0.)]


In [ ]:
# 制約条件を守っているかどうかの情報を得ることができる
decoded_samples = model.decode_sampleset(sampleset=sampleset, feed_dict=feed_dict)
for sample in decoded_samples:
  print(sample.constraints(only_broken=True))

{}
{}
{'constr_1': (False, 2.0)}
{'constr_1': (False, 2.0)}
{'constr_1': (False, 2.0)}
{'constr_1': (False, 1.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 1.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 1.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 3.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 2.0), 'constr_2': (False, 2.0)}


In [ ]:
# コスト値：上から1つ目の出力結果を整形
# A→B→E→D→C→A
sampleset.record[0][0].reshape(N, N)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]], dtype=int8)

# Openjijでの実行

In [ ]:
!pip install openjij

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 716 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 


In [ ]:
from openjij import SQASampler
sampler = SQASampler()

In [ ]:
sampleset = sampler.sample_qubo(qubo, num_reads=10)
print(sampleset.record)

[([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -5930., 1)
 ([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], -5990., 1)
 ([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], -9910., 1)
 ([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], -7930., 1)
 ([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], -9890., 1)
 ([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], -7940., 1)
 ([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], -9910., 1)
 ([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], -9890., 1)
 ([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], -5960., 1)
 ([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], -7930., 1)]


In [ ]:
# コスト値：上から3番目の出力結果を整形
# A→B→E→D→C→A
sampleset.record[2][0].reshape(N, N)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]], dtype=int8)